# NOTEBOOK 2 : Fine-tuning de RoBERTa pour la Classification de Dépôts

**Objectif :** Ce notebook entraîne un classifieur basé sur `roberta-base` pour assigner automatiquement des catégories aux dépôts GitHub.
1.  Il charge le CSV des dépôts et la base de catégories (JSON) créée dans le Notebook 1.
2.  Il prépare un jeu de données étiquetées en assignant à chaque dépôt la catégorie la plus proche sémantiquement.
3.  Il divise les données en un jeu d'entraînement (80%) et un jeu de test (20%).
4.  Il fine-tune le modèle `roberta-base` sur les données d'entraînement.
5.  Il évalue la performance du modèle sur le jeu de test avec plusieurs métriques.

In [ ]:
!pip install pandas numpy torch transformers datasets scikit-learn sentence-transformers accelerate

In [ ]:
import pandas as pd
import numpy as np
import json
from tqdm.notebook import tqdm

# Hugging Face
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)

# Métriques
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer


# --- CONFIGURATION ---

# Fichiers d'entrée
INPUT_CSV_FILE = "github_data_with_readmes.csv"
CATEGORIES_FILE = "github_categories_database.json"

# Modèles
BASE_MODEL = 'roberta-base' # Le modèle que nous allons fine-tuner
EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2' # Pour l'étiquetage initial

# Paramètres
OUTPUT_MODEL_DIR = "./roberta_github_classifier" # Dossier où sauvegarder le modèle entraîné
TEST_SIZE = 0.2 # 20% des données pour le test

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Utilisation du device : {device}")

In [ ]:
# Chargement des données des dépôts
df = pd.read_csv(INPUT_CSV_FILE)
df['description'] = df['description'].fillna('')
df['readme_content'] = df['readme_content'].fillna('')
df['full_text'] = df['description'] + ' ' + df['readme_content']
df = df[df['full_text'].str.strip().str.len() > 50].reset_index(drop=True)

# Chargement de la base de catégories
with open(CATEGORIES_FILE, 'r', encoding='utf-8') as f:
    categories_db = json.load(f)

# Création d'un mapping ID -> Nom pour plus tard
id2label = {cat['category_id']: cat['category_name'] for cat in categories_db}
label2id = {v: k for k, v in id2label.items()}
N_LABELS = len(categories_db)

print(f"{len(df)} dépôts et {N_LABELS} catégories chargés.")

In [ ]:
print("Création des étiquettes pour l'entraînement supervisé...")

# 1. Re-générer les embeddings des dépôts (ou les sauvegarder/recharger depuis le Notebook 1)
embedding_model = SentenceTransformer(EMBEDDING_MODEL, device=device)
repo_embeddings = embedding_model.encode(
    df['full_text'].tolist(), 
    show_progress_bar=True,
    batch_size=64
)

# 2. Récupérer les embeddings prototypes de nos catégories
category_embeddings = np.array([cat['embedding_prototype'] for cat in categories_db])

# 3. Calculer la similarité et assigner le label le plus proche à chaque dépôt
print("Assignation de la catégorie la plus proche à chaque dépôt...")
similarity_matrix = cosine_similarity(repo_embeddings, category_embeddings)
# `argmax` nous donne l'index (et donc l'ID) de la catégorie avec le plus haut score de similarité
df['label'] = np.argmax(similarity_matrix, axis=1)

# On renomme la colonne de texte pour plus de clarté
df.rename(columns={'full_text': 'text'}, inplace=True)

print("Jeu de données étiquetées prêt.")
df[['full_name', 'label']].head()

In [ ]:
# Division en jeux d'entraînement et de test
train_df, test_df = train_test_split(df, test_size=TEST_SIZE, random_state=42, stratify=df['label'])

# Conversion en objets Dataset de Hugging Face
train_dataset = Dataset.from_pandas(train_df[['text', 'label']])
test_dataset = Dataset.from_pandas(test_df[['text', 'label']])

# Création d'un DatasetDict
hf_datasets = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

print(hf_datasets)

# Tokenisation
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

tokenized_datasets = hf_datasets.map(tokenize_function, batched=True)

In [ ]:
# Chargement du modèle pré-entraîné
model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL, 
    num_labels=N_LABELS,
    id2label=id2label,
    label2id=label2id
)

# Fonction pour calculer les métriques pendant l'évaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# Définition des arguments d'entraînement
training_args = TrainingArguments(
    output_dir=OUTPUT_MODEL_DIR,
    num_train_epochs=3,              # 3 époques est un bon point de départ
    per_device_train_batch_size=16,  # Réduire si vous manquez de VRAM
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy="epoch",     # Évaluer à la fin de chaque époque
    save_strategy="epoch",           # Sauvegarder à la fin de chaque époque
    load_best_model_at_end=True,     # Charger le meilleur modèle à la fin
)

# Création de l'objet Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

# Lancement du fine-tuning
print("Lancement du fine-tuning...")
trainer.train()

In [ ]:
print("\nÉvaluation du modèle final sur le jeu de test...")
eval_results = trainer.evaluate()

print("\n--- RÉSULTATS DE L'ÉVALUATION ---")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

# Sauvegarde du meilleur modèle et du tokenizer
trainer.save_model(OUTPUT_MODEL_DIR)
tokenizer.save_pretrained(OUTPUT_MODEL_DIR)
print(f"\n✅ Modèle fine-tuné et tokenizer sauvegardés dans '{OUTPUT_MODEL_DIR}'")

In [ ]:
# Chargement du pipeline avec notre modèle local fine-tuné
classifier = pipeline("text-classification", model=OUTPUT_MODEL_DIR, device=0 if device == 'cuda' else -1)

# Exemple de description de dépôt
new_repo_description = """
A new JavaScript library for creating interactive and beautiful data visualizations using D3.js and React. 
It supports various chart types like bar, line, and pie charts.
"""

# Faire une prédiction
prediction = classifier(new_repo_description)

print("\n--- TEST SUR UN NOUVEL EXEMPLE ---")
print(f"Description: '{new_repo_description.strip()}'")
print(f"Catégorie prédite : {prediction[0]['label']} (Score: {prediction[0]['score']:.4f})")